Forecast model for Walmart's top 10 best-selling products using an LSTM based on detailed specifications for cross-validation, train-test splits, and sequence handling

In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import TimeSeriesSplit

In [2]:
# Load data
df = pd.read_csv('../raw_data/cleaned_merge_df_top10.csv')
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)

In [3]:
# Prepare data for LSTM
scaler = MinMaxScaler(feature_range=(0, 1))
df['scaled_sales'] = scaler.fit_transform(df[['sales']])

def create_sequences(data, input_length, output_length):
    X, y = [], []
    for i in range(len(data) - input_length - output_length + 1):
        X.append(data[i:(i + input_length)])
        y.append(data[(i + input_length):(i + input_length + output_length)])
    return np.array(X), np.array(y)

input_length = 28
output_length = 28
n_features = 1  # since we are only using sales as feature
n_splits = 10  # Number of folds


In [9]:
# Time Series Cross-validation
tscv = TimeSeriesSplit(n_splits=n_splits)

# Define the LSTM model outside the loop
model = Sequential([LSTM(50, activation='relu'), Dense(output_length)])
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])


In [10]:
# Training and evaluation
fold_results = []

for train_index, test_index in tscv.split(df):
    train, test = df.iloc[train_index], df.iloc[test_index]

    # Create sequences
    X_train, y_train = create_sequences(train['scaled_sales'].values, input_length, output_length)
    X_test, y_test = create_sequences(test['scaled_sales'].values, input_length, output_length)

    # Reshape for LSTM input
    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], n_features))
    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], n_features))

    # Fit the model
    model.fit(X_train, y_train, epochs=10, batch_size=16, verbose=0)

    # Evaluate the model
    predictions = model.predict(X_test)
    predictions = scaler.inverse_transform(predictions)
    y_test = scaler.inverse_transform(y_test)

    mae = mean_absolute_error(y_test, predictions)
    fold_results.append(mae)

53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [11]:
# Report results
print("MAE per fold:", fold_results)
print("Average MAE:", np.mean(fold_results))

# Forecast next 28 days for the last sequence of the last fold
last_sequence = df['scaled_sales'].values[-input_length:]
last_sequence = last_sequence.reshape((1, input_length, n_features))
future_sales = model.predict(last_sequence)
future_sales = scaler.inverse_transform(future_sales)
print("Forecast for the next 28 days:", future_sales.flatten())

MAE per fold: [21.19958665151798, 17.015310540892976, 20.560944233249753, 16.436080863669687, 18.358872259324592, 15.655364271159211, 13.379781732088892, 11.81029666768563, 10.772487540574614, 11.49169107084056]
Average MAE: 15.668041583100386
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
Forecast for the next 28 days: [19.430387  20.779943  63.081856  40.621223  44.707546  25.435373
 41.890106  24.857689  24.929724   3.1096044 11.904546  18.456148
 53.09849   29.418821  30.64181   19.387747  34.28429   20.550407
 23.340523   3.4327629  8.233639  13.101522  50.576714  26.018362
 25.010578  15.743027  30.641811  15.393671 ]
